In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold

In [ ]:
dim = 3
n_gauss = 3
n_pts_per_gauss = 300

centers = np.zeros((n_gauss,dim))
for i in range(1,n_gauss):
    centers[i] = np.random.randint(0,2,3)
    
print(centers)

cov_m = [np.diag([0.01 for i in range(dim)]),np.diag([0.1 if i%2 !=0 else 0.1 for i in range(dim)])]

D = np.zeros((n_pts_per_gauss*n_gauss,dim))
c = np.zeros(n_pts_per_gauss*n_gauss)
for i in range(dim):
    k = np.random.randint(0,2,1)[0]
    D[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = np.random.multivariate_normal(centers[i],cov_m[k],n_pts_per_gauss)
    c[i*n_pts_per_gauss:(i+1)*n_pts_per_gauss] = i
    
D = (D-np.min(D,axis=0))/(np.max(D,axis=0)-np.min(D,axis=0))

In [ ]:
%matplotlib qt
fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(projection='3d')
ax.scatter(D[:,0], D[:,1], D[:,2])

In [ ]:
t_sne = manifold.TSNE(
    n_components=2,
    perplexity=30,
    init="random",
    random_state=0,
)

S = t_sne.fit_transform(D)

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(S[:,0],S[:,1],c=c)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

In [ ]:
# Define the MLP model
class NNinv(nn.Module):
    def __init__(self, input_size, output_size):
        super(NNinv, self).__init__()
        
        # Define the layers
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),  # Input to first hidden layer
            nn.ReLU(),
            nn.Linear(64, 128),  # First hidden layer to second hidden layer
            nn.ReLU(),
            nn.Linear(128, 256),  # Second hidden layer to third hidden layer
            nn.ReLU(),
            nn.Linear(256, 512),  # Third hidden layer to fourth hidden layer
            nn.ReLU(),
            nn.Linear(512, output_size),  # Fifth hidden layer to output
            nn.Sigmoid()  # Output layer with sigmoid activation
        )
    
    def forward(self, x):
        return self.layers(x)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(S, D, test_size=0.33, random_state=42)

In [ ]:
print(X_train.shape,y_train.shape)

In [ ]:
# Example usage
input_size = 2  # Example input size (can be changed)
output_size = dim   # Binary classification (sigmoid output for single output)
model = NNinv(input_size, output_size)

# Create DataLoader for batch processing
batch_size = 64
t_X_train = torch.tensor(X_train)
t_y_train = torch.tensor(y_train)
dataset = TensorDataset(t_X_train, t_y_train)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Instantiate the model, loss function, and optimizer
model = NNinv(input_size, output_size)
loss_fn = nn.L1Loss()  # Mean Absolute Error (MAE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Number of epochs to train
num_epochs = 5

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(dataloader):
        # Forward pass
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    # Print the average loss for the epoch
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

print("Training complete.")

In [ ]:
t_X_test = torch.tensor(X_test)
t_y_test = torch.tensor(y_test)
outputs_test = model(t_X_test)
loss_test = loss_fn(outputs_test, t_y_test)
print(loss_test/y_test.shape[0])